## 법정동, 행정동 코드매핑 자료 다운로드 및 저장
#### '행정기관(행정동) 및 관할구역(법정동) 변경내역' 파일(가장 최신파일) 다운로드, 저장
###### https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardList.do?bbsId=BBSMSTR_000000000052

압축 파일 중, mdm.city 테이블 생성을 위해 **KIKmix** 로 시작하는 파일만 사용합니다. (법정동/ 행정동 코드 모두 나와 있는 자료)

In [1]:
#라이브러리 설치

import pandas as pd
import os

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [2]:
#파일 경로를 설정 및 파일 읽기
# path = 'C:/Users/user/Desktop/인수인계/인수인계_데이터/행정동_법정동/jsocde20230611/KIKmix.20230611.xlsx'
file_path = 'D:\PythonProject\data-gatherer\statistics\data\jscode20230703\jscode20230703\KIKmix.20230703.xlsx'
#업데이트 날짜 기록하기 위해 파일 경로의 일부를 추출
basename = os.path.basename(file_path)      # 파일명 + 확장자
filename = os.path.splitext(basename)[0]    # 확장자를 뺀 파일명
update_date = filename.split(".")[1]
print("업데이트 일자 : ",update_date)      

업데이트 일자 :  20230703


In [3]:
conn = dbconnect.db_connect("DEMO_DM")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM city WHERE 업데이트일자 = {update_date})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [4]:
if data_exist == 1:      # 데이터 존재하면
    print(f'{update_date} 일자에 데이터가 존재합니다.')

20230703 일자에 데이터가 존재합니다.


### 파일의 데이터 읽기

In [23]:
df_code = pd.read_excel(file_path)
df_code.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [28]:
#법정동, 행정동을 나타낸 테이블 생성 > 대구, 경산시, 고령군, 동명면에 한정
df_daegu_table = df_code[df_code['시도명'].str.contains('대구광역시')]
df_gyeongsan_table = df_code[df_code['시군구명']=='경산시']
df_goryeong_table = df_code[df_code['시군구명']=='고령군']
df_dongmyeong_table = df_code[df_code['읍면동명']=='동명면']

df_code_table = pd.concat([df_daegu_table,df_gyeongsan_table,df_goryeong_table,df_dongmyeong_table],axis = 0) #데이터 결합
df_code_table.reset_index(inplace= True, drop=  True)
df_code_table['업데이트일자'] = update_date

# NaN 값을 공백으로 변환 (Table에 저장하기 위함)
df_code_table['시군구명'] = df_code_table['시군구명'].fillna('')
df_code_table['읍면동명'] = df_code_table['읍면동명'].fillna('')

#순서 변경
df_code_table=df_code_table[['업데이트일자','행정동코드','시도명','시군구명','읍면동명','법정동코드','동리명']]
df_code_table['행정동코드'] =df_code_table['행정동코드'].apply(str)
df_code_table['법정동코드'] =df_code_table['법정동코드'].apply(str)
df_code_table.head()

,업데이트일자,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명
0,20230703,2700000000,대구광역시,,,2700000000,대구광역시
1,20230703,2711000000,대구광역시,중구,,2711000000,중구
2,20230703,2711051700,대구광역시,중구,동인동,2711010100,동인동1가
3,20230703,2711051700,대구광역시,중구,동인동,2711010200,동인동2가
4,20230703,2711051700,대구광역시,중구,동인동,2711010300,동인동3가


### 데이터를 테이블에 저장
형식을 지정한 후 **mdm.city** 테이블에 데이터를 삽입해 줍니다.  
mdm.city는 법정동, 행정동 관련 정보가 있는 테이블 입니다.

In [29]:
conn = dbconnect.db_connect("DEMO_DM")
cur = conn.cursor()

for row in df_code_table.itertuples():
    sql = "insert into city (업데이트일자, 행정동코드, 시도명, 시군구명, 읍면동명, 법정동코드, 동리명) \
           values (%s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7]))

conn.commit()
conn.close()